# PMSM Halbach rotor

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import sys, os
sys.path.append(os.path.relpath("../ngsolve/src"))
import hiped as hp
from ngsolve import *
from ngsolve.webgui import Draw
from IPython.display import clear_output
from copy import deepcopy

# 1) Geometry

In [ ]:
from geometry import poleMotor

Npp=4
mesh = poleMotor(Npp, maxh = 0.004)
Draw(mesh)

# 2) Material definition

In [ ]:
Br = 1
magnetsBr = []
Th = np.linspace(0,2*pi, 2*Npp+1)
for i in range(len(Th)):
    magnetsBr.append( lambda b, th=Th[i] : CF((cos(th), sin(th))) )
magnetsBr = magnetsBr[:-1]   

# 3) Interpolation of physical properties

In [ ]:
domain = hp.Domain(2*Npp) 

# Remanent polarization

penalBr = hp.Penalization("simp", 1, reverse=False)
childBr = [hp.VertexFunction(label = "M"+str(i), f = br , dfdu = lambda u : CF(((0,0),(0,0)), dims = (2,2)),
                             flagNGSolve = True) for i, br in enumerate(magnetsBr)]
interpBr =  hp.Interpolation(domain, childBr, label = "rotor", penalization= penalBr)

# definition of rho
fesRho = L2(mesh, definedon = "Rotor")
rho = interpBr.setInitialVariable(typeInit = "zero", radius = 2, NGSpace = fesRho) 
rho = interpBr.projection(rho)
plt.figure()
interpBr.plot(rho)
plt.show()

# 4) Solver

In [ ]:
nu0 = 1/(4e-7*pi)
R = CF(((0,1),(-1,0)), dims = (2,2))

# since the problem is linear and only the RHS changes we can assemble and invert the matrix only one
fes = Periodic(H1(mesh, dirichlet = "a0"),[-1,-1,-1])
a, aStar = fes.TnT()
bf = BilinearForm(fes)
bf += grad(aStar) * nu0 * grad(a) * dx
bf.Assemble()
Minv = bf.mat.Inverse(fes.FreeDofs())

def solveState(rho):
    lf = LinearForm(fes)
    lf +=  - nu0* grad(aStar) * (R * interpBr.eval(rho, R*grad(a))) *  dx("Rotor")
    lf.Assemble()
    gfu = GridFunction(fes)
    gfu.vec.data = Minv * lf.vec
    return gfu
           
u = solveState(rho)
Draw(u, mesh)

# 4) Objective function and adjoint

In [ ]:
areaAirgap = Integrate(CF(1) * dx("Airgap"), mesh)
perimeterAirgap = Integrate(CF(1) * ds("e1"), mesh)
thicknessAirgap = areaAirgap/perimeterAirgap

ur = Normalize(CF((x,y)))
R = CF(((0,1),(-1,0)), dims = (2,2))

def Jobj(a):
    return Integrate((R*grad(a))*ur*dx("Airgap"), mesh)/thicknessAirgap

def dJobj(aStar):
    return (R*grad(aStar))*ur*dx("Airgap")

def solveAdjoint(a,rho):
    fes = Periodic(H1(mesh, dirichlet = "a0"), [-1,-1,-1])
    p, aStar = fes.TnT()
    rhs = LinearForm(dJobj(aStar))
    rhs.Assemble()
    gfu = GridFunction(fes)
    gfu.vec.data = -1* Minv * rhs.vec
    return gfu

p = solveAdjoint(u,rho)
Draw(p)

# 5) Gradient of the objective function

In [ ]:
def gradRho(u,p,rho):
    b = Norm(grad(u))
    w, dwdx = interpBr.evalBasisFunction(rho)
    dBrdrho = interpBr.evaldx(rho,u, w)
    g = rho.copy()
    keys = w.keys()
    for k in keys:
        g[k] = [ R * dBrdrho[k][i] * nu0 * grad(p) for i in range(len(dBrdrho[k]))]
    return g

In [ ]:
## Initialization

alpha = 0.1       # Initial step
alpha_min = 1e-4  # Minimal step
n_max = 1000      # Maximum number of iterations
n = 0

objectiveHistory = [Jobj(u)]
rhoHistory = [deepcopy(rho)]

In [ ]:
## Optimization loop
recompute = True
while( n < n_max and alpha > alpha_min):
    
    if recompute:
        # 1) State :
        directState = solveState(rho)
    
        # 2) Adjoint :
        adjointState = solveAdjoint(directState,rho)
    
        # 3) Gradient computation :
        gradient = gradRho(directState,adjointState,rho)
        normG = GridFunction(fesRho)
        normG.Set(sqrt(gradient['rotor'][0]**2 +gradient['rotor'][1]**2) )
    
    # 4) Update :
    rho_test = deepcopy(rho)
    for i in range(len(rho['rotor'])):
        rho_test['rotor'][i].Set(rho['rotor'][i] - alpha * gradient['rotor'][i]/normG) 
    n += 1
    
    # 5) Projection :
    
    rho_test = interpBr.projection(rho_test)
    
    # 6) Step size control :
    a_test = solveState(rho_test)
    objectiveHistory.append(Jobj(a_test))
    clear_output(wait = True)
    
    print(f'it n°{n} | f = {objectiveHistory[-1]} | step = {alpha}')
    
    if objectiveHistory[-1] >= objectiveHistory[-2]:
        alpha = alpha/2
        objectiveHistory.pop()
        recompute = False
    elif objectiveHistory[-1] < objectiveHistory[-2]:
        alpha = alpha*1.2
        rho = rho_test
        rhoHistory.append(deepcopy(rho))
        recompute = True

In [ ]:
w, dwdx = interpBr.evalBasisFunction(rhoHistory[-1])
nMag = 5
Draw(w['rotor'][nMag]*magnetsBr[nMag](1), mesh, vectors={"grid_size" : 20, "offset" : 0.5 })

In [ ]:
plt.figure()
plt.plot(objectiveHistory)
plt.show()

In [ ]:
plt.figure()
interpBr.plot(rhoHistory[-1])
plt.show()